# 2D Diffusion Equation Solver

This file contains the code for obtaining a discretized matrix representation of 2D diffusion equation via Finite Volume Method

In [1]:
# Importing Solver functions from jupyter file
%run iterative_solvers.ipynb
import numpy as np

In [2]:
# Defining  main solver function

def two_D_DEsolver(x_pos, y_pos, D_mesh, abs_mesh, source_mesh, solver="sor", err_tol=1.e-6):
    """
    x_pos: position of material along x direction (n+1 by 1)
    y_pos: position of material along y direction (m+1 by 1)
    D_mesh: Diffusion constant distribution over meshes (m by n)
    abs_mesh: Absorption macroscopic cross section distribution over meshes (m by n)
    source_mesh: Fixed source distribution over meshes (m by n) 
    solver: jacobi, gs and sor are available solvers (default is sor)
    err_tol: error tollerance used by numerical solver
    """
    m = len(y_pos)-1
    n = len(x_pos)-1
    matrix_A = np.zeros(((m+1)*(n+1),(m+1)*(n+1)))
    phi_solutions = np.empty(((m+1)*(n+1),1))
    phi_solutions.fill(np.nan)
    abs_list = phi_solutions.copy()
    
    Left=np.empty((n*(m+1),1))
    Left.fill(np.nan)
    Right=np.empty((n*(m+1),1))
    Right.fill(np.nan)
    Bottom=np.empty((m*(n+1),1))
    Bottom.fill(np.nan)
    Top=np.empty((m*(n+1),1))
    Top.fill(np.nan)
    Center=np.empty(((m+1)*(n+1),1))
    Center.fill(np.nan)
    
    #Absorption: Non corner or edge points    

    for j in reversed(range(1,m)): #the position starts from bottom left corner
        for i in range(1,n):
            d1=np.abs(x_pos[i]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            d2=np.abs(x_pos[i+1]-x_pos[i])
            e2=np.abs(y_pos[j-1]-y_pos[j])

            V1=0.25*d1*e1
            V2=0.25*d2*e1
            V3=0.25*d2*e2
            V4=0.25*d1*e2
            
            abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j,i]*V2+abs_mesh[j-1,i]*V3+abs_mesh[j-1,i-1]*V4
            source_value = source_mesh[j,i-1]*V1+source_mesh[j,i]*V2+source_mesh[j-1,i]*V3+source_mesh[j-1,i-1]*V4
            abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1] = abs_value
            phi_solutions[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]= source_value
            
    #Absorption: Reflecting Side Right Points
    for j in reversed(range(1,m)): 
        i=n
        d1=np.abs(x_pos[i]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        e2=np.abs(y_pos[j-1]-y_pos[j])
        
        V1=0.25*d1*e1
        V4=0.25*d1*e2

        abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j-1,i-1]*V4
        source_value = source_mesh[j,i-1]*V1+source_mesh[j-1,i-1]*V4
        abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1] = abs_value
        phi_solutions[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]= source_value       

    #Absorption: Reflecting Side top Points
    for i in range(1,n): 
        j=0
        d1=np.abs(x_pos[i]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        d2=np.abs(x_pos[i+1]-x_pos[i])
        
        V1=0.25*d1*e1
        V2=0.25*d2*e1

        abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j,i]*V2
        source_value = source_mesh[j,i-1]*V1+source_mesh[j,i]*V2
        abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1] = abs_value
        phi_solutions[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]= source_value   
        
    #Absorption: Top Right
    i,j = n,0
    d1=np.abs(x_pos[i]-x_pos[i-1])
    e1=np.abs(y_pos[j]-y_pos[j+1])
    V1=0.25*d1*e1
    abs_value = abs_mesh[j,i-1]*V1
    source_value = source_mesh[j,i-1]*V1
    abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1] = abs_value
    phi_solutions[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]= source_value
    
    
    #Flux: Reflecting side top
    for i in range(1,n): #the position starts from bottom left corner
        j=0
        d1=np.abs(x_pos[i]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        d2=np.abs(x_pos[i+1]-x_pos[i])
        #e2=np.abs(y_pos[j-1]-y[j])

        a_L=-(D_mesh[j,i-1]*e1)/(2*d1)
        a_R=-(D_mesh[j,i]*e1)/(2*d2)
        a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
        a_C=abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]-(a_L+a_R+a_B)

        Left[(m+1-(j+1))*(n+1-1)+i-1]=a_L
        Right[((m+1-(j+1))*(n+1-1)+i)]=a_R
        Bottom[(m+1-(j+1)-1)*(n+1)+i]=a_B
        Center[(m+1-(j+1))*(n+1)+i]=a_C
        
    #Flux: Reflecting side right
    for j in reversed(range(1,m)): #the position starts from bottom left corner
        i=n
        d1=np.abs(x_pos[i]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        e2=np.abs(y_pos[j-1]-y_pos[j])

        a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
        a_B=-(D_mesh[j,i-1]*d1)/(2*e1)
        a_T=-(D_mesh[j-1,i-1]*d1)/(2*e2)
        a_C=abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]-(a_L+a_B+a_T)

        Left[((m+1-(j+1))*(n+1-1)+i-1)]=a_L
        Bottom[(m+1-(j+1)-1)*(n+1)+i]=a_B
        Top[(m+1-(j+1))*(n+1)+i]=a_T
        Center[(m+1-(j+1))*(n+1)+i]=a_C 
        
    #Flux: Non corner or edge points
    for j in reversed(range(1,m)): #the position starts from bottom left corner
        for i in range(1,n):
            d1=np.abs(x_pos[i]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            d2=np.abs(x_pos[i+1]-x_pos[i])
            e2=np.abs(y_pos[j-1]-y_pos[j])
            
            a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
            a_R=-(D_mesh[j,i]*e1+D_mesh[j-1,i]*e2)/(2*d2)
            a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
            a_T=-(D_mesh[j-1,i-1]*d1+D_mesh[j-1,i]*d2)/(2*e2)
            a_C=abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]-(a_L+a_R+a_B+a_T)
            
            Left[((m+1-(j+1))*(n+1-1)+i-1)]=a_L
            Right[((m+1-(j+1))*(n+1-1)+i)]=a_R
            Bottom[(m+1-(j+1)-1)*(n+1)+i]=a_B
            Top[(m+1-(j+1))*(n+1)+i]=a_T
            Center[(m+1-(j+1))*(n+1)+i]=a_C       
   
    #Flux: Top right
    i,j = n,0
    d1=np.abs(x_pos[i]-x_pos[i-1])
    e1=np.abs(y_pos[j]-y_pos[j+1])


    a_L=-(D_mesh[j,i-1]*e1)/(2*d1)
    a_B=-(D_mesh[j,i-1]*d1)/(2*e1)
    a_C=abs_list[len(y_pos)*len(x_pos)-(j+1)*len(x_pos)+(i+1)-1]-(a_L+a_B)

    Left[((m+1-(j+1))*(n+1-1)+i-1)]=a_L
    Bottom[(m+1-(j+1)-1)*(n+1)+i]=a_B
    Center[(m+1-(j+1))*(n+1)+i]=a_C

    for i in range((m+1)*(n+1)):
        matrix_A[i,i]=Center[i]
    
    for i in range(len(Top)):
        matrix_A[i,i+(n+1)*(m+1)-len(Top)]=Top[i]
        matrix_A[i+(n+1)*(m+1)-len(Bottom),i]=Bottom[i]

    A_Index=0
    V_Index=0
    for i in range((m+1)*(n+1)):
        skip = (i+1)%(n+1)
        if skip!=0: 
            matrix_A[A_Index,A_Index+1]=Right[V_Index]
            matrix_A[A_Index+1,A_Index]=Left[V_Index]
            A_Index+=1
            V_Index+=1
        else:
            A_Index+=1

    for i in range(len(phi_solutions)):
        if np.isnan(phi_solutions[i]):
            phi_solutions[i]=0
            matrix_A[i,:]= np.zeros((1,len(matrix_A[i,:])))
            matrix_A[i,i]=1
            
    guess = np.ones(((m+1)*(n+1),1))/np.linalg.norm(np.ones(((m+1)*(n+1),1)),2)
    
    # Using imported solver functions
        # using SOR, jacobi and Gauss to arrive at 3 solutions
    if solver == "gs":
        iterations, fluxes = gauss_sedidel_solver(np.array(matrix_A), np.array(phi_solutions).flatten(), np.array(guess).flatten() , err_tol)
    elif solver == "jacobi":
        iterations, fluxes = jacobi_solver(np.array(matrix_A),np.array(phi_solutions).flatten(), np.array(guess).flatten() , err_tol)
    else:
        iterations, fluxes = sor_solver(np.array(matrix_A),np.array(phi_solutions).flatten(), np.array(guess).flatten() , err_tol)
    
    return iterations, fluxes